# Lab 02 · HTTP Forms and Retry

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- A robust fetch helper with retry logic is introduced.
- A controlled React form is configured with loading and error feedback.
- Friendly error messages are surfaced in the UI.

## What will be learned
- Retry helpers are structured for frontend HTTP calls.
- Controlled form patterns in React are rehearsed.
- Error boundaries in simple forms are practiced.

## Prerequisites & install
The following commands are intended for local execution.

```bash
cd ai-web/frontend
npm install
```

### FastAPI service context

The frontend exercises in this lab interact with a FastAPI application structured around routers, request models, and response classes documented in the [FastAPI docs](https://fastapi.tiangolo.com/). Each route wraps an async function that receives validated data—such as prompt text or user metadata—via `Pydantic` models, raising typed `HTTPException`s when validation fails. Dependency-injected services handle downstream AI calls or persistence, while middleware centralizes logging, CORS rules, and auth. The automatic OpenAPI schema and interactive docs remain useful for confirming the exact payload shape your retry helper must send.

For AI-focused endpoints you will often:
- Separate inference, embedding, and health routes into dedicated router modules.
- Return structured responses that include generation metadata (latency, token counts) so the UI can surface insights.
- Wrap third-party model clients in try/except blocks that translate provider errors into FastAPI `HTTPException`s with informative status codes.

### Vite-powered React environment refresher

The React frontend is scaffolded with [Vite](https://vitejs.dev/guide/), giving you fast hot module replacement, first-class TypeScript support, and `import.meta.env` configuration for secrets like AI API keys or backend URLs. React components (see the [React documentation](https://react.dev/)) encapsulate form state, loading indicators, and streaming responses. Utility modules centralize fetch logic—such as the retry helper you extend here—so components stay declarative.

When layering AI UX patterns consider:
- Co-locating UI state (input text, response history) with hooks that call the backend.
- Surfacing background work (retry attempts, exponential backoff timers) through toasts or progress bars.
- Using Suspense-like fallbacks or skeleton components when streaming responses from long-running AI calls.

### AI request flow checklist

1. Define or review the FastAPI router handling the AI request, ensuring request/response models match the expected form fields.
2. Confirm the route's error handling returns consistent status codes and messages that the frontend retry helper can inspect.
3. Expose the backend base URL via `.env` values so Vite can proxy requests locally and bundle the correct production endpoint.
4. Build or update a shared API utility (e.g., `withRetry(fetchAIResponse)`) that reads from `import.meta.env` and emits structured results.
5. Connect React form components to the helper, wiring loading states, disabled buttons, and retries to maintain a responsive AI UX.
6. Test the round trip locally using Vite's dev server with proxying to FastAPI before moving to containerized builds.


## Step-by-step tasks
### Step 1: Retry helper placement
A retry helper is positioned under src/lib.

In [ ]:
from pathlib import Path
lib = Path("ai-web/frontend/src/lib")
lib.mkdir(parents=True, exist_ok=True)
(lib / "retry.js").write_text('''export async function withRetry(fn, attempts = 2, delayMs = 400) {
  let lastError;
  for (let attempt = 0; attempt <= attempts; attempt += 1) {
    try {
      return await fn();
    } catch (error) {
      lastError = error;
    }
    await new Promise((resolve) => setTimeout(resolve, delayMs));
  }
  throw lastError;
}
''')
print("Retry helper was written.")

### Step 2: Form integration
The React form now leans on a reusable `withRetry` helper so network hiccups do not derail the student experience. The flow looks like this:

1. Collect user input in local component state.
2. Call `withRetry` with an async function that posts the form data.
3. Retry the call if a transient failure occurs, backing off briefly between attempts.
4. Show a friendly message if every attempt fails, otherwise render the echoed response.

```jsx
const sendEcho = () =>
  withRetry(() => post('/echo', { msg }), 2, 500);
```

Because `withRetry` only depends on an async callback, you can scale the pattern across modules. For example, a profile page could share the helper from `src/lib/retry.js` and dial in its own policy:

```jsx
const fetchProfile = () =>
  withRetry(() => api.getProfile(userId), 3, 800);
```

> Tip: pass higher attempt counts and delays when hitting slower services or when layering exponential backoff (`delayMs * attempt`).

Larger forms follow the same controlled-input structure: hold each field in state, validate before submitting, and surface field-level errors next to the relevant inputs. When multiple fields need to submit together, build one payload object and reuse `withRetry` so the entire submission benefits from resilience.

To see retries in action, the backend now exposes `/flaky-echo`. Point the helper at it during testing:

```jsx
await withRetry(
  () => post('/flaky-echo?failures=2', { msg }),
  3,
  500,
);
```

The first two requests return HTTP 503 errors, the third succeeds, and the UI recovers gracefully. Wrap longer-lived retry sequences in loading spinners, keep buttons disabled until the promise settles, and consider error boundaries to catch truly fatal issues. These guardrails help the codebase grow while staying aligned with the lab’s resilient frontend architecture.

In [ ]:
from pathlib import Path
app_js = Path("ai-web/frontend/src/App.jsx")
text = app_js.read_text()
if "withRetry" not in text:
    text = text.replace(
        "import { post } from './lib/api';",
        "import { post } from './lib/api';
import { withRetry } from './lib/retry';",
    )
if "withRetry" in text and "withRetry(() => post" not in text:
    text = text.replace(
        "const json = await post('/echo', { msg });",
        "const json = await withRetry(() => post('/echo', { msg }), 2, 500);",
    )
if "setError(String(err));" in text:
    text = text.replace(
        "setError(String(err));",
        "setError('A temporary issue was encountered. Please try again.');",
    )
app_js.write_text(text)
print("App.jsx was adjusted for retry and friendly errors.")

## Validation / acceptance checks
```bash
# locally
curl -X POST http://localhost:8000/echo -H 'Content-Type: application/json' -d '{"msg":"retry"}'
```
- The echoed payload is returned successfully after transient failures are simulated.
- React development mode shows the described UI state without console errors.

## Homework / extensions
- Additional retry backoff strategies are outlined for future reference.
- Form validation rules are drafted to prevent empty submissions.